In [1]:
# Script to gather VASP input files and modify parameters for KPOINTS and INCAR
# Requires locally saved POSCAR file, and reference KPOINTS and INCAR files
# POTCAR file is generated by appending POTCAR files (on server) for each element
# Server side .sbatch file is utilized

from pyExecVaspSlurm import Job, Server

/Users/dominicwelti/opt/anaconda3/lib/python3.9/site-packages/paramiko/transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [ ]:
# Initialize Job
job = Job('MD_ML_Test_MRB2_12_550eV_3x5x6_bulk_4c',poscar_path_local='/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/VASP_reference_files/Ex_4b_poscar_beta_LPS')

job.incar_load('/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/VASP_reference_files/incar_MD_ML')
job.kpoints_load('/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/VASP_reference_files/kpoints_reference')

In [ ]:
# Modify INCAR and KPOINTS
job.incar_modify('ENCUT','550')
job.incar_modify('ISMEAR','0') # Gaussian smearing
job.incar_modify('SIGMA','0.2')
job.incar_modify('PMASS','1000')
job.kpoints_modify('subdivisions along recipr. l.', '3 5 6')

In [ ]:
# Initialize Server
server = Server('<username>','<hostname>', '<homepath>', '<path to sbatch file for submitting jobs>') # all refer to server
sftp, ts = server.connect()

In [ ]:
# Check k-point mesh
job.kpoints

In [ ]:
# Check INCAR settings
job.incar

In [ ]:
# Modify INCAR settings
job.incar_modify('ML_LMLFF','.TRUE.')

In [ ]:
# As an example for evaluating various temperatures for an ab-initio MD run: upload and start jobs with various values for TEBEG in INCAR
import pandas as pd

parameters = pd.DataFrame({
    'TEBEG': ['900', '1100', '1200', '850'],
})

for tebeg in parameters['TEBEG']:
    job.incar_modify('TEBEG',tebeg)
    k_points = job.kpoints.loc[3,"Value"].replace(' ', 'x')
    e_cut = float(job.incar.loc[job.incar['Setting'] == 'ENCUT','Value'].values[0])
    job.name = f'MD_Test_TEBEG_{tebeg}k_{k_points}_{int(e_cut)}eV_bulk_4b'
    print(job.name, tebeg)

    server.create_folder(sftp, job.name)
    server.potcar_gather(job, sftp, '<e.g. homepath/mypps/potpaw_PBE>')
    server.incar_upload(job, sftp, '/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/VASP_temporary')
    server.poscar_upload(job,sftp)
    server.kpoints_upload(job,sftp,'/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/VASP_temporary')
    server.name_batch(job, ts) # rename job in sbatch file
    server.batch_submit(job, ts)

In [ ]:
# Terminate connection
if sftp: sftp.close()
if ts.active: ts.close()